# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

## Please add your own API key in api_keys.py ## 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("./weather_csv", encoding="utf-8")
del city_weather_df["Unnamed: 0"]
city_weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nikolskoye,59.7035,30.7861,28.00,100,90,4.47,RU,1612028757
1,Vaini,-21.2000,-175.2000,79.88,78,100,14.65,TO,1612028563
2,Nizwá,22.9333,57.5333,61.92,41,0,1.61,OM,1612028758
3,Bathsheba,13.2167,-59.5167,84.20,61,40,14.97,BB,1612028758
4,Upernavik,72.7868,-56.1549,-9.69,86,2,8.21,GL,1612028758
...,...,...,...,...,...,...,...,...,...
549,Binzhou,37.3667,118.0167,43.00,50,100,6.26,CN,1612028879
550,Salinas,36.6777,-121.6555,52.00,76,90,14.97,US,1612028823
551,Camacupa,-12.0167,17.4833,65.55,93,100,5.55,AO,1612028879
552,Zhaozhou,45.6833,125.3167,-17.05,87,33,6.49,CN,1612028880


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

lat_lon = city_weather_df[["Latitude", "Longitude"]]
humidity = city_weather_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_city = city_weather_df[(city_weather_df["Max Temp"] < 80) 
                & (city_weather_df["Max Temp"] > 70) 
                & (city_weather_df["Wind Speed"] < 10) 
                & (city_weather_df["Cloudiness"] == 0)].dropna()
ideal_city

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Ouadda,8.0777,22.4007,79.36,22,0,6.89,CF,1612028759
49,Hermanus,-34.4187,19.2345,71.01,77,0,5.99,ZA,1612028564
76,Vangaindrano,-23.3500,47.6000,78.60,85,0,7.81,MG,1612028774
126,Bredasdorp,-34.5322,20.0403,71.01,77,0,5.99,ZA,1612028785
168,Lima,-12.0432,-77.0282,77.00,64,0,8.05,PE,1612028711
213,Oddur,4.1213,43.8894,78.08,20,0,8.84,SO,1612028804
220,Palāsa,18.7667,84.4167,70.32,77,0,2.48,IN,1612028805
244,Alibag,18.6411,72.8792,77.00,38,0,7.14,IN,1612028811
266,Gadag,15.4167,75.6167,73.78,31,0,7.02,IN,1612028816
372,Grand Gaube,-20.0064,57.6608,79.00,81,0,4.00,MU,1612028840


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_city[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
locations = hotel_df[["Latitude", "Longitude"]]



In [6]:
params = {"radius": 5000, 
         "types": "lodging", 
         "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params).json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    if index == 13:
        print(name_address)
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing fields")

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing fields
{
    "html_attributions": [],
    "next_page_token": "ATtYBwLTFJCTobUXtvAFrEekBjWfkWt3E-s0krdMp8lBxwby132ie10Q_xZkbmeC_CRBOIa5uFKDqWEsExn9Dh4-8H3fs6Qu_OoL04Cl71BGij0N1DoFkmyg7RWdpnQrNJpit6bMBdKDIiyuxMcLEApccc1TYlu18SxBxV9OVAiSxB1d5q_kXjo2nTDxFF4wmzmLhm519ZSdL3QEge2SQ8pe86ww56PWWvTZm93qHkR-mhA5r1FaGtvohKNuT-P9rUNnRrGwXhFFkmxingSq8SuRWtqoODRJXqlFnjr5gfI2aI-_uI63nmMTDLckdyq8BudXX_F0sLlEHzvsz91pSxzgecUrtJvtu9cBqB6_ty_rb-U-tJiH26s24j4s8Y-94sQ4JjtASKsDkRztdwFoRKiWmgNLReswVX7llSCm9pODSvILWYgrfp4rf9MLd8b_Y6h58cUj2S_GPP6vl--QZWNCgGcu2XYzu5ni0ja2DbDe5-v8WTuXY4NSRlohp7f3-5R8ipVDkfdDSR3slDq7sNKshcMk0g9tYDK6ukv0EWK-iJnoFgfBQ05kueum8Dh4lJI67ID1jkGlGMqaZWDXppXB1F2ydpB5cqhvl6ExAObM6TkbF5Su_5QjXj30wl8hUcoScFWYKGCtFqQ6S37nr4m_iQzyIf335r18hxvDpxIPQPV1zwyFB11tMszi25TmsUt5J_5WZcXqn4jzzJqUTukug-iPU_QNaQ3fOUOnIPGA0fQb",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -23.3484693,
                    "lng": 47.60643059999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -23.34670196970849,
                        "lng": 47.60741748029149
                    },
                    "southwest": {
                        "lat": -23.3493999302915,
                        "lng": 47.60471951970849
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel Fanilo",
            "place_id": "ChIJNU9GQIilxCERUv0oBc6IFKo",
            "plus_code": {
                "compound_code": "MJ24+JH Vangaindrano, Madagascar",
                "global_code": "5H89MJ24+JH"
            },
            "r

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJmoeuVNd9-UrXBA7vGgDhLp7YMaw1o-Q4mvtBLNFDv2I51qrw8XORoEikk1tzTkQRm1h8sgBEme5Re9hovIAF5Rzmnfe3EqgFoRXCp0BZnsay0HBMoPOEYe1z5qCnbBNGoZBK5sFS1Y0m4TIzasefjHa_EYPABrRnM4AQLC_tmma2gKl_wQwR_7irGMP1VVT_vwB0XB7dpsrsEOkYs0ef7A1x-2Vy0HL3QQOY-MbY7umyn0wUVrAClGH-Ic_DuAKe-RJt4tyRFlg2VXkw6LSSPTrVbLxBoLnCJq4w2UBzHUQMLSSzgh6lTYtZ6pfn_KI0POYyoztrJthrJWsrMOK8ybfHUs3bu9tRzrJtGNJXa4bm3BY2xngUTXJ2tYiBxAUtt8pIN-BETC28olMcArNqFmND5oidiQIsZfSQmW7ZoY2nzevjN8bVapL7sen52-qXeUxslKV8yXhs2cxMRywjGlc8rn1mCzp0ajXIww5iyuGrWCyZGJ9BlyVMUE7gavqtfupQ1U1Aeo6mn0QzZ6ma7h1gGvrcNRYLP7LN0kciD56WQudXtLLMKUqLHh46WbFtRxNQYZkuAHKrn-dN2k05QbB3l_cQ_P6ZR6SGWv4yDqiEytG1tUPKRv2YMON6ROXKDP_8o8zKgfor6gFc9tPUKmBu2VktK_KFtrwtrxadNr8__DrE6D8dDir9b6_AsveCeToHRi9sCizkqmj-ltZproq8hAL5PKcJwezj5plKe",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -12.0509609,
                    "

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 4.121543900000001,
                    "lng": 43.8890408
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.122826130291504,
                        "lng": 43.89039613029149
                    },
                    "southwest": {
                        "lat": 4.120128169708499,
                        "lng": 43.88769816970849
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Banjanay Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2280,
                    "html_attributions": [
                        "

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJe3Ag6Xhw9Aw7v02BI-UIllsrKlSTnh03u30T6v2Lm-DGGkHSR1j6yS8PD7rNchgA7d2u4WJHeUNJ1RyGT1uwXWb55osOwMSw7HAhK-MvmvG55E1iTgO-91LNgEd8et4rp3CazgDmizi3m8k02-4jnjmYrhrE2JWeUCl4EgeLhUe7fSipRlBoURehSlWjkNX7GmsEDxuPcYT5CXPdZ36QxBxEEGDUO1Adz8VNanCEpUc-SxwilP23sguyXkOHxG0kA__tt_MsDbqwDFIQUG7s9ZY9C4eTnntJQIJTLQ8dc2RLA9g76Cr7YISfhEzmJYzfEDZ8d7BteMjcNs-CYsmzRD2LGSySJ2BpE4wLESsLC_Ei3M6pIXY37z73aZDy0N1kJutjL7_97NDHK-12XHadyL4ziU-3wk81O0ZNPYobjBjCoElHKSD-FqSyOOad6Rwe2_W_MFPL5tq7GAPduKjMwGhq1nGfWydOqo86pBBNlrUMux7XpFQXFkpBnu1llDhBrK2jqpvTcptkXXMHsC6j_WC3AZEjldyVjtO2ODG0aqPh4I5Riem3z7NiRcENJnpmj8hP-z87j_OC_ge0ikUqAY4Iw0yesSI0Sa7CZAG7XGJF9f5eTy4xAZHyKhOwfadjqwBDJhxQdUyrR1Y3mp0UHvRBUs0oTWvoDL95QPYvMAryoKuNfDCG6tLqMQaA9wsHL3dJyc7dwDwJc-KStrewbIWPfk6QL7_ywcLIOSdm5",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 18.7673327,
                    "l

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJkpM5JZ4_o03_zyh8zlbthJEz81ytlMW05UDkCYBh6O_8O-jZaxc6_zcHr4TjhYUj5jy1ArIIxAkRB4Ua_RLsw1KLNoUkXVmogFozL9bbwPNopsCvoThKsp5IM_kHTdh_IulG6ELSua0vps-qYcJkzIAo9ouf5u-UxTTvTyXXq0Fosl1ac8A07w92R9dShZ6lQBT4f7gHCM_iqYXoRbUPCh6ZBibilV67ZTbF7RscdL7h0yOYk3uzhyvagUFnPuPzijmEMsWHlB82xa0KMa1t-hn-sxEqzxz6xjOox33-Zi69BBDNrX3FeiZgI0VurVHjNtNz_HPTk_cueMoX-k5CZDi0sho4ul2kfh8QB-KNqCFx2zx5YI2oAyhpRouok7CFg2-tdU0Y0pfEGMPUnhb_hHmcWrwuXy5NeakfIYsCMCDINX08H7NyKQhFzA0kddaBgG8KsKb5NdjAc9iHt9lQFFkIdvBoOqlBt2KFqKW3C-ETYohfVjGtWfZZ1Jq9efU_ZJxbiXfQBoXq8nITCvxSp317qWVUaYSNXP-O7RDjIlfdEY_4yIM76uDvtxPa5tPGDgaa0rnT6x8fqKR5pWW6b4DgDeJ8d_PKx9xm8aStlriTkxGVeke0YgKyeLEeLKyS7kzg7B47urnq9cxkytj5Wv6s6CpTFpiSKfem-Nw60_rMITLrxkbzKDrwjNPLGB1tQwHZdkucmmwi-tTC7QrKZiqAeAS8pmAV_JK9Do-tH",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 15.4220386,
                    "l

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL9fXfSTPtay6iL3LWPyHH_tbaRxBv791qW1hUcM9pb5jzvpkWEd7RwiPq7l4dike9fHq8MwYIC0S2DlqwMspvunnJMSnpqO7cqAAbS5J-C_N_O4mllWeBIMowaze_CtI14rRQCWRW7jE42MZOVE4-5hIuSw_NRaBUPigQ6jhsQ42bjBaHZ14Inr3qod5xkxDxJrZKOAk5SEM7i8x_vXmvEtLSKWRU-blFqAQjkSRbuQqnV5Klm4SDMZKebuxZbv5Tav8al6zXAU3xcuDeBTfa7L2ifD_Mzfa9iBE3dSQrKsmTQtqnOMsGctUMreO72CuXZGTPKyXN8f69dXO42kZMtKxZWA0PfwUEJG9D21A4v1DEt5miq__GQQsisqecwlORYziezL3g2iEcwQSVYJzHm-b0xht3y-eu9JOnqto4OjB3srX3hILFP_WZiFEsdSm-u4GJ-QxfUH0scHkwR14dVoANlwqFf91h8ZBX2eXkLKegy9RHezHngqmGipmH6WZbNNX-SS1_fXnrU42nVdQxq3FE7kyzM58e18TAuvWIIeoqoACcNBZOGGFPAVySuqo6Hh1EBCrcjz6Ct4rU2Uv2Lmr6xcqIqFyymCdIPPuDbjnTpaBn3MKwUQcYNeB2rHgUbVcb8Q8trrT9cGlVAPm3L3KdUXGZ2dpw4uRxG8O0xINdNyw5HzcYKN4mPrgyAv2DYF1VwoT9yVJlh7okKnoyiC2wK30b-0A11CZITfb7l",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.0065691,
                    "

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKvxt1rmwiSaE0GLrJJC3d70kSILc4Jemh5sOqRPhJRWhVhFPopp2gBWbwZzznjR3KoWUuay2cJDqrOc_9um8DPpwMhqzXjQbDUTBw2uxoey-2NcHXD0BmP4LNpjEdSML-KZCZ4qdnavAu9NkESXoiWB04cfTPRwiv6t3G0a5w_D7cXmxSVHC32OaBS2--SBpMeqxYIJd8DlNRfCDsDGwkm7Ycj1O6F-YFfF3hYH6W00A_NmrU5eLMhuhUFqLaU7gmvlDxvJA_5tXfNgUrji-m_MUhgUegXbShh59PUf_idLybOsscI6gIN4R7G7HskdREkE6Xn2m8TG1V_YEkEg-umQXz5IOg9kVvAqZr4GmW3T_jTWjY5-prkyjS7g-pTxmEVOFQTu1yd_Fwy47wiKxson_ffx1PTYCkF-5ihosJhmIcsx2tTcOZbUVP3KihFpLxAbxHm_L8J0sEnYxlk0AdSQVm6ycz3vVBtjEWXIEWS_8ebnqh85dZMQiOTVEZ0GzGWaFKCxwREPfLy8oIhYvPNFn2lBme1dhCgEG_WE03Sjcfv2wyEKOvbtGZ_I1Uk-yjtgwQRtSLCAzz6jRKLfMI_iyEWpAs5Zct5sRIv7TDvLY8L1lgpQnIeGwpEMeKFHNC90aU30MlUjlkBByTe7HJwImQlX4YJlK0kMJj-MfczbETlBDnZaFccO6ERvNlo7w1ZY4jwmjrkWKllqnz-qeq-GYoCYtCuvv8IOQvFp8sV",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 18.1344858,
                    "l

{
    "html_attributions": [],
    "next_page_token": "ATtYBwILrkhS8HS0-Iu3BAHfs87U2F6CfYvx-G16B78TgoFHfmR-32cAkWYuHgNtljy3nXDGvev6EVrwRabqksm5jWLYjM3w7BbapODnIdU5_VaFfIGweGKnacLoVL8OcaENDj6BjpPC_H7kY1o1iNfMa4W4JvcTXlmSulYbR7oTl2am-PaHLojUPTa5fcjOmR7IXLRp3mHUOc9mBxxtYj5QaQTq4X2-PWV8ZFpAQpc7xPBHjvRBh9kiBwrM_KAF0eaptXAZdSEto8Aj8uTnc5Uq1IJs-Th60p4q1eR-KUbuhOleishW7hCDaNItyvPhSLvws5ei1j2pK3q9yrkCOhQirgpNhg_akLGqnkca1E8stiLkV7Fr26Fp9WmZ5Dp-QJqOKRsNSzrEJqHRGXoVWIQWlV_Qq-4g5eS9IhBxLIv2pHROrutYSyPtDbahNgMSo4S7kABVhsH_nbJ7CIIDvyXxJ7g-22ta4NpM26IfHK7qrPTlgkU__HCcIkbAaOY9VzFfJ7whEQcerUtHhYrwrH5MlH1ZnIavI6pgOHF28VhanmO9FWMF6lCL_sNlXLlSvrQQTa7VYgWLZ5ohAZnewgw1GJY7__PovWlPkIUGerT6nRfkpvvSRsQl0v4Q4Zhx106CUwksrYYgEo5keviXba9cmRPi-2MTVWaYJuR5ilpJG9xnEq4_OCwv7MHe2tcHOOyn7qGtED_Tqwq1Cz9klWeV1KBoi3W84uiPLkugcX1Y7xq9",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -17.6549048,
                    "

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLG3ggeTCvo1UXWoJGQc65wqaBIvdCZGC1KgbQWGa3UJsNPmmbrfiij6VigXL1w65rdGFQFHW7dPnwfU4r6L_i9mTf310JxlXcl7On5C4LE-W1-rLQcWt2t19SdC0J1lN7aZIuTT3OjogzR3JhACnUDIPGA7X76XmbCWVXC28XCt0663J6AE4GS4Nhhv3WiXhnmqFYffzMBK1-aPNRUFdAdOCDa6CArgaPy3DcoWeyqyT1BdU5zLXoZwH3s_9jyoGAlyHluIuCQIJ0wOrbfL4yoTonKkcpxHp-rl74NhdrCBsNwVf0qi9rinQSVkbboO6ojLDNICsXdZ5UKUtfPunplSJYTdWwH8EaeVQlglIYqh6-QTS6g2YvjkXQgnANuipgbbfGVSV8YJlkF9jZj-07ceYR6BI_gxs2FcjYYK2seWBzZ-KdxwnhTjxwjO8nOxCxmecoP9v4y_Bx-nYmSkJlRUqPbJu_O0y5rHsmrdhIKlkdmYKPjy0iOeVnOsIkccc2UnGwTtvZyPQv7W4Sz2zk7G0J9IWxh7M8AHFol0TMLoPC3FBDtBGWsBRes4h15qF39hpQzLnJ2P8O7eH2S6l3t8zZZORhrxgvYTplqQoW9Rt4yVS5Io0kWtoodxoBbrFdH2cOWddBWi7oqIxEbFB8NBP3I29Bk4Be2436Tkl8tA9EZQHQ71IXtXlMDJ2mIZuYQ-nXaqlIcwRSvP3IeDK2Y2rzOKozMH5_gfLFfz81W",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 12.8083829,
                    "l

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
hotel_df